In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import wandb

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
X_train, Y_train, X_test, Y_test = torch.load('../data/dataset_2_40D.pt')

# 将数据移动到适当的设备
X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)

# 使用 DataLoader 进行批处理
l = 64
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=l, shuffle=True)

D = 40
m = 100

class PM_Euler(nn.Module):
    def __init__(self, input, hidden_layer, output):
        super(PM_Euler, self).__init__()
        self.relu = nn.ReLU()
        self.hidden_dim = hidden_layer
        self.W = nn.Parameter(torch.rand(input, hidden_layer, device=device), requires_grad=True)
        # HE初始化
        nn.init.kaiming_normal_(self.W, mode='fan_in', nonlinearity='relu')
        self.a = nn.Parameter(torch.rand(hidden_layer, output, device=device), requires_grad=True)
        nn.init.kaiming_normal_(self.a, mode='fan_in', nonlinearity='relu')
        
    def forward(self, x):
        # print(x.shape)
        z1 = self.relu(torch.mm(x, self.W))
        # print(z1.shape)
        z2 = torch.mm(z1, self.a) / self.hidden_dim
        return z2

    def loss(self, y_pred, y_true):
        return (y_pred - y_true.reshape(y_pred.shape)) ** 2

model = PM_Euler(D + 1, m, 1).to(device)

# 计算模型W和a的Norm
def get_norm(model):
    return torch.norm(model.W).item(), torch.norm(model.a).item()

get_norm(model)

/tmp/ipykernel_4124/24256467.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train, Y_train, X_test, Y_test = torch.load('../data/dataset_2_40D.pt')


(9.18452262878418, 15.034234046936035)

In [4]:
epochs = 1000
lr = 0.01
_lambda = 0

train_losses = []
test_losses = []

import datetime

wandb.config = {
    'learning_rate': lr,
    'batch_size': l,
    'epochs': epochs,
    'hidden_layer': m,
    'input': D + 1,
    '_lambda': _lambda,
    'output': 1,
    'optimizer': 'ESAV'
}

date = datetime.datetime.now().strftime("%m%d%H%M")
wandb.init(project='Numerical Method', name=f"PM_ESAV_Example_2_{date}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pheonizard (pheonizard-university-of-nottingham). Use `wandb login --relogin` to force relogin


In [5]:
for epoch in range(epochs):
    cnt = 0
    for X, Y in train_loader:
        loss = model.loss(model(X), Y).mean()
        if cnt == 0:
            r = torch.exp(loss)
            cnt = 1
        loss.backward()
        with torch.no_grad():
            theta_w_1 = model.W.data
            theta_a_1 = model.a.data
            N_a = model.a.grad.clone()
            N_w = model.W.grad.clone()
            b_a = r * N_a / torch.exp(loss)
            b_w = r * N_w / torch.exp(loss)
            model.a.data = (theta_a_1 - lr * b_a) / (1 + lr * _lambda)
            model.W.data = (theta_w_1 - lr * b_w) / (1 + lr * _lambda)
            #=========Update SAV R================
            r = torch.exp(torch.log(r) + torch.sum(b_a * (model.a - theta_a_1)) + torch.sum(b_w * (model.W - theta_w_1)))
            model.a.grad.zero_()
            model.W.grad.zero_()
    with torch.no_grad():
        train_loss = model.loss(model(X_train), Y_train).mean()
        test_loss = model.loss(model(X_test), Y_test).mean()
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        norm = get_norm(model)
        wandb.log({'epoch': epoch + 1,
                   'train_loss': train_loss, 
                   'test_loss': test_loss,
                   'norm_W': norm[0],
                   'norm_a': norm[1],
                   'accuracy': 1 - test_loss,
                   'r': r.item()})
        print(f'epoch {epoch + 1}, loss {train_loss:.8f}, test loss {test_loss:.8f}')

epoch 1, loss 0.98125762, test loss 0.99613708
epoch 2, loss 0.95925117, test loss 0.97376740
epoch 3, loss 0.93316162, test loss 0.94733727
epoch 4, loss 0.90335923, test loss 0.91723359
epoch 5, loss 0.87864077, test loss 0.89227396
epoch 6, loss 0.85274249, test loss 0.86610037
epoch 7, loss 0.83129144, test loss 0.84445620
epoch 8, loss 0.80475718, test loss 0.81767201
epoch 9, loss 0.78585368, test loss 0.79861981
epoch 10, loss 0.76609695, test loss 0.77874976
epoch 11, loss 0.74205768, test loss 0.75456864
epoch 12, loss 0.72018766, test loss 0.73258400
epoch 13, loss 0.70324159, test loss 0.71554542
epoch 14, loss 0.68112457, test loss 0.69333488
epoch 15, loss 0.66028059, test loss 0.67237687
epoch 16, loss 0.63789946, test loss 0.64985669
epoch 17, loss 0.61891931, test loss 0.63074332
epoch 18, loss 0.60189390, test loss 0.61359453
epoch 19, loss 0.58347315, test loss 0.59501749
epoch 20, loss 0.56551361, test loss 0.57689738
epoch 21, loss 0.54667544, test loss 0.55788684
e

In [15]:
wandb.finish()

accuracy,▁▂▃▄▄▅▆▆▇▇▇█████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
norm_W,▁▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇▇▇▇▇████████████████████
norm_a,▁▁▁▂▂▃▃▄▄▅▅▆▆▆▇▇▇▇▇▇████████████████████
r,██▃▆▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▇▆▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9332
epoch,264
norm_W,13.3976
norm_a,16.68236
